In [ ]:
# Install required packages if needed
# !pip install openai python-dotenv langchain faiss-cpu pandas

from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import faiss
import pickle
import os

class RAGSystem:
    def __init__(self, embedding_model="text-embedding-3-small", completion_model="gpt-4o-mini"):
        """Initialize the RAG system with OpenAI models."""
        self.client = OpenAI()
        self.embedding_model = embedding_model
        self.completion_model = completion_model
        self.documents = []
        self.index = None
        self.document_store = {}
        
    def add_documents(self, documents: List[str], metadatas: List[Dict[str, Any]] = None):
        """Add documents to the RAG system and create embeddings."""
        if metadatas is None:
            metadatas = [{} for _ in range(len(documents))]
            
        # Store original documents with their metadata
        start_idx = len(self.documents)
        for i, (doc, meta) in enumerate(zip(documents, metadatas)):
            idx = start_idx + i
            self.document_store[idx] = {"content": doc, "metadata": meta}
            self.documents.append(doc)
        
        # Generate embeddings
        embeddings = self._get_embeddings(documents)
        
        # Create or update the FAISS index
        if self.index is None:
            self._create_index(embeddings)
        else:
            self._update_index(embeddings)
            
        print(f"Added {len(documents)} documents. Total documents: {len(self.documents)}")
    
    def _get_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for the given texts using OpenAI."""
        embeddings = []
        for text in texts:
            response = self.client.embeddings.create(
                model=self.embedding_model,
                input=text
            )
            embeddings.append(response.data[0].embedding)
        return np.array(embeddings, dtype=np.float32)
    
    def _create_index(self, embeddings: np.ndarray):
        """Create a new FAISS index with the embeddings."""
        vector_dimension = len(embeddings[0])
        self.index = faiss.IndexFlatL2(vector_dimension)
        self.index.add(embeddings)
    
    def _update_index(self, new_embeddings: np.ndarray):
        """Update the existing FAISS index with new embeddings."""
        self.index.add(new_embeddings)
    
    def retrieve(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """Retrieve relevant documents based on the query."""
        # Generate query embedding
        query_embedding = self._get_embeddings([query])[0].reshape(1, -1)
        
        # Search for similar documents
        distances, indices = self.index.search(query_embedding, top_k)
        
        # Get the retrieved documents
        results = []
        for i, idx in enumerate(indices[0]):
            if idx < len(self.documents) and idx >= 0:
                doc_info = self.document_store[idx]
                results.append({
                    "content": doc_info["content"],
                    "metadata": doc_info["metadata"],
                    "score": float(distances[0][i])
                })
        
        return results
    
    def generate(self, query: str, top_k: int = 3, temperature: float = 0.7) -> str:
        """Generate a response using RAG approach."""
        # Retrieve relevant documents
        retrieved_docs = self.retrieve(query, top_k=top_k)
        
        # Construct the context from retrieved documents
        context = "\n\n".join([f"Document {i+1}:\n{doc['content']}" for i, doc in enumerate(retrieved_docs)])
        
        # Create the augmented prompt
        system_prompt = f"""You are a helpful assistant. Use the following retrieved documents to answer the user's question. 
If you cannot answer the question based on the documents, say so.

Retrieved documents:
{context}"""
        
        # Generate completion with the augmented prompt
        response = self.client.chat.completions.create(
            model=self.completion_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ],
            temperature=temperature
        )
        
        return response.choices[0].message.content
    
    def save(self, path: str = "rag_system"):
        """Save the RAG system to disk."""
        os.makedirs(path, exist_ok=True)
        
        # Save the documents and document store
        with open(os.path.join(path, "document_store.pkl"), "wb") as f:
            pickle.dump(self.document_store, f)
            
        with open(os.path.join(path, "documents.pkl"), "wb") as f:
            pickle.dump(self.documents, f)
            
        # Save the FAISS index
        if self.index is not None:
            faiss.write_index(self.index, os.path.join(path, "index.faiss"))
            
        print(f"RAG system saved to {path}")
    
    @classmethod
    def load(cls, path: str = "rag_system", embedding_model="text-embedding-3-small", completion_model="gpt-4o-mini"):
        """Load a saved RAG system from disk."""
        rag = cls(embedding_model=embedding_model, completion_model=completion_model)
        
        # Load the documents and document store
        with open(os.path.join(path, "document_store.pkl"), "rb") as f:
            rag.document_store = pickle.load(f)
            
        with open(os.path.join(path, "documents.pkl"), "rb") as f:
            rag.documents = pickle.load(f)
            
        # Load the FAISS index if it exists
        index_path = os.path.join(path, "index.faiss")
        if os.path.exists(index_path):
            rag.index = faiss.read_index(index_path)
            
        print(f"RAG system loaded from {path}")
        print(f"Total documents: {len(rag.documents)}")
        
        return rag

# Usage example
if __name__ == "__main__":
    # Create a new RAG system
    rag = RAGSystem()
    
    # Example documents about travel destinations
    documents = [
        "Hawaii is known for its stunning beaches, volcanic landscapes, and rich Polynesian culture. The islands offer activities like surfing, snorkeling, and hiking. Best time to visit is between March and September.",
        "Paris, France is famous for the Eiffel Tower, Louvre Museum, and exquisite cuisine. The city is ideal for art lovers, romantics, and food enthusiasts. Best time to visit is April to June or October to November.",
        "Tokyo, Japan blends traditional culture with futuristic technology. Visitors can explore ancient temples, enjoy cherry blossoms in spring, and experience world-class shopping and dining. Best time to visit is March-April or October-November.",
        "Santorini, Greece features iconic white buildings with blue domes overlooking the Aegean Sea. The island is perfect for couples seeking romantic sunsets, beaches, and Mediterranean cuisine. Best time to visit is April to October.",
        "Bali, Indonesia offers lush rice terraces, spiritual temples, and vibrant beach resorts. Popular activities include surfing, yoga retreats, and traditional dance performances. Best time to visit is April to October."
    ]
    
    # Add documents to the RAG system
    rag.add_documents(documents)
    
    # Example query
    query = "What's a good destination for a beach vacation?"
    
    # Generate a response using RAG
    response = rag.generate(query)
    print(f"Query: {query}\n")
    print(f"Response: {response}")